In [5]:
import os
from openai import OpenAI
from pykrx import stock
import pandas as pd

MODEL = "gpt-4o-mini"
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_stock_code_by_name(name: str) -> str:
    tickers = stock.get_market_ticker_list()
    for ticker in tickers:
        if stock.get_market_ticker_name(ticker) == name:
            return ticker
    raise ValueError(f"종목명 '{name}'에 해당하는 종목코드를 찾을 수 없습니다.")

def fetch_stock_price(stock_name: str, start: str, end: str) -> pd.DataFrame:
    code = get_stock_code_by_name(stock_name)
    df = stock.get_market_ohlcv_by_date(start, end, code)
    df.reset_index(inplace=True)
    df = df[["날짜", "시가", "고가", "저가", "종가", "거래량"]]
    return df

def convert_to_text(df: pd.DataFrame) -> str:
    text = ""
    for _, row in df.iterrows():
        text += f"{row['날짜'].strftime('%Y-%m-%d')}: 시가 {row['시가']:,}, 고가 {row['고가']:,}, 저가 {row['저가']:,}, 종가 {row['종가']:,}, 거래량 {row['거래량']:,}\n"
    return text

def analyze_stock(text_data: str) -> str:
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "다음은 한국 주식시장에서 특정 종목의 날짜별 주가 데이터입니다. 추세, 특징적인 변동, 이상치, 투자자 입장에서 인사이트를 분석해주세요."},
            {"role": "user", "content": text_data}
        ],
        temperature=0.5,
        max_tokens=1024
    )
    return response.choices[0].message.content

def run_stock_analysis(stock_name: str, start: str, end: str):
    print(f"📈 '{stock_name}' 주가 데이터 ({start}~{end}) 조회 및 분석 시작")
    df = fetch_stock_price(stock_name, start, end)
    text_data = convert_to_text(df)
    print("📝 GPT에게 보낼 데이터 예시:\n", text_data[:800], "\n...(중략)...")
    analysis = analyze_stock(text_data)
    print("\n📊 GPT 분석 결과:\n")
    print(analysis)

if __name__ == "__main__":
    run_stock_analysis("에이피알", "2024-12-01", "2025-01-10")


📈 '에이피알' 주가 데이터 (2024-12-01~2025-01-10) 조회 및 분석 시작
📝 GPT에게 보낼 데이터 예시:
 2024-12-02: 시가 49,250, 고가 51,000, 저가 48,300, 종가 48,950, 거래량 287,940
2024-12-03: 시가 48,950, 고가 53,400, 저가 48,900, 종가 52,600, 거래량 830,198
2024-12-04: 시가 51,000, 고가 52,900, 저가 49,850, 종가 51,100, 거래량 549,582
2024-12-05: 시가 52,400, 고가 54,900, 저가 51,800, 종가 53,600, 거래량 903,700
2024-12-06: 시가 54,900, 고가 55,000, 저가 50,700, 종가 53,800, 거래량 721,156
2024-12-09: 시가 52,200, 고가 53,700, 저가 50,200, 종가 50,500, 거래량 437,926
2024-12-10: 시가 50,900, 고가 54,300, 저가 50,900, 종가 53,900, 거래량 629,303
2024-12-11: 시가 55,900, 고가 58,400, 저가 53,500, 종가 54,900, 거래량 1,341,236
2024-12-12: 시가 55,600, 고가 56,700, 저가 53,800, 종가 55,300, 거래량 1,934,684
2024-12-13: 시가 55,000, 고가 55,900, 저가 53,500, 종가 53,800, 거래량 520,692
2024-12-16: 시가 53,900, 고가 55,400, 저가 52,900, 종가 54,300, 거래량 390,099
2024-12-17: 시가 54,100, 고가 54,200, 저가 52,500, 종가  
...(중략)...

📊 GPT 분석 결과:

주어진 날짜별 주가 데이터를 분석하여 추세, 특징적인 변동, 이상치, 그리고 투자자 입장에서의 인사이트를 정리하겠습니다.

### 1. 추세 분석
- **상승세**: 2024년 12